In [3]:
from nsepy import get_history 
import datetime 
from datetime import date
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
from nsepy.derivatives import get_expiry_date 

Table = pd.read_csv('Optionpivot.csv')
Table['month'] = pd.to_datetime(Table['Date']).dt.strftime('%m')

Table

,Date,Nifty,Type,Expiry,9000,9100,9200,9300,9400,9500,...,11900,12000,12100,12200,12300,12400,12500,12600,12700,month
0,2018-04-02,10264.00,CE,2018-04-26,1249.95,1158.00,1051.10,956.05,857.95,763.25,...,0.70,0.75,0.65,NaN,NaN,NaN,NaN,NaN,NaN,04
1,2018-04-02,10264.00,CE,2018-05-31,NaN,1463.05,1020.40,1273.15,1180.05,806.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04
2,2018-04-02,10264.00,CE,2018-06-28,NaN,NaN,994.90,1228.95,1415.90,780.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04
3,2018-04-02,10264.00,PE,2018-04-26,3.70,5.00,6.55,8.05,11.10,14.50,...,1700.45,1725.50,848.30,NaN,NaN,NaN,NaN,NaN,NaN,04
4,2018-04-02,10264.00,PE,2018-05-31,NaN,20.85,26.75,31.70,39.90,49.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,2018-12-24,10681.90,PE,2018-12-27,NaN,0.25,0.80,0.90,0.95,1.00,...,1108.45,1311.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
978,2018-12-26,10748.50,CE,2018-12-27,NaN,1630.20,1521.15,1319.35,1333.55,1246.45,...,0.40,0.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
979,2018-12-26,10748.50,PE,2018-12-27,NaN,0.25,0.25,0.30,0.35,0.30,...,1143.05,1249.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
980,2018-12-27,10783.65,CE,2018-12-27,NaN,1689.20,1576.45,1465.35,1364.65,1280.05,...,0.05,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12


In [8]:
Table1 = Table.groupby('month').first()
Table1['M'] = pd.to_datetime(Table1['Date']).dt.strftime('%m')
Table1['ATM_Strike'] = round(Table1['Nifty']/100)*100

Table3 = pd.DataFrame({"Date":['0'], "Close":['0'], "Expiry":['0'], "Type":['0'], "month":['0'], 
                       "ATM":0, "OTM":0, "ITM":0, "Butterfly":0, "P&L":0})

Table4 = pd.DataFrame({"Date":['0'], "Close":['0'], "Expiry":['0'], "Type":['0'], "month":['0'], 
                       "ATM":0, "OTM":0, "ITM":0, "Butterfly":0, "P&L":0})
Table4
for x in range (0,9):
    e=Table1.M[x]
    d=Table1.Expiry[x]
    a= int((Table1.ATM_Strike[x]))
    b = a+200
    c = a-200
    f1 = str(a)
    f2 = str(b)
    f3 = str(c)
    Table2 = pd.DataFrame({"Date": Table["Date"], "Close": Table["Nifty"], "Expiry": Table["Expiry"], 
                           "Type": Table["Type"], "month": Table["month"], "ATM": Table[f1], 
                           "OTM": Table[f2], "ITM": Table[f3]})
    Table2= Table2.where((Table2.month == e) & (Table2.Expiry == d) & (Table2.Type == "CE"))
    Table2 = Table2.dropna()
    Table2['Butterfly']=(Table2['ATM']*2)-Table2['ITM']-Table2['OTM']
    Table2['P&L']=Table2['Butterfly'].diff().cumsum()
    Table3 = Table3.append(Table2)
    Table4= Table4.append(Table2.where((Table2.Date == d)))
    Table4 = Table4.dropna()

Table4.drop(["Date", "Close", "Expiry", "Type", "ATM", "OTM", "ITM", "Butterfly"], 
            axis = 1, inplace = True)
Table4['Total_P&L']=Table4['P&L'].cumsum()
print (Table4)

,Date,Close,Expiry,Type,month,ATM,OTM,ITM,Butterfly,P&L
0,0,0,0,0,0,0,0,0,0,0


In [6]:
Table3.to_csv("Daywise_strategy_computation.csv", index=True, encoding='utf8')